In [1]:
from rastrigin import *

In [37]:
n_exec = 33
fitness = list()
pc = list()
pm = list()
f_l = list()
f_m = list()

In [3]:
mut_probs = np.arange(start = 0.010, stop = 0.055, step = 0.005)

for mut_prob in mut_probs:
    print(mut_prob)
    for i in range(n_exec):
        GA = GeneticAlgorithm(n = 10, m = 20, l = 20, max_eval = 10000, mut_prob = mut_prob, cross_prob = 0.75, k = 20)
        GA.solve()
        fitness.append(GA.best_fitness)
        print(GA.best_fitness)
    pm.append(fitness.copy())
    fitness.clear()

Generations:   1%|          | 70/10000 [00:00<00:14, 691.68it/s]

0.01


Generations:   1%|          | 66/10000 [00:00<00:15, 652.66it/s]

16.05744454722273


Generations:   1%|          | 74/10000 [00:00<00:13, 724.17it/s]

...

11.569410007440709


Generations:   1%|          | 90/10000 [00:00<00:11, 890.04it/s]

5.848725804737981


Generations: 100%|██████████| 10000/10000 [00:11<00:00, 864.58it/s]

10.41297561635119


In [4]:
best_mut_prob = mut_probs[np.argmin(np.mean(pm, axis = 1))]

In [14]:
cross_probs = np.arange(start = 0.6, stop = 0.95, step = 0.05)

for cross_prob in cross_probs:
    print(cross_prob)
    for i in range(n_exec):
        GA = GeneticAlgorithm(n = 10, m = 20, l = 20, max_eval = 10000, mut_prob = best_mut_prob, cross_prob = cross_prob, k = 20)
        GA.solve()
        fitness.append(GA.best_fitness)
        print(GA.best_fitness)
    pc.append(fitness.copy())
    fitness.clear()

Generations:   1%|          | 70/10000 [00:00<00:14, 696.17it/s]

0.6


Generations:   1%|          | 78/10000 [00:00<00:12, 767.60it/s]

2.4046957049673807


Generations:   1%|          | 64/10000 [00:00<00:15, 638.57it/s]

...

2.3305205140553795


Generations:   1%|          | 76/10000 [00:00<00:13, 752.75it/s]

9.687795732290681


Generations: 100%|██████████| 10000/10000 [00:12<00:00, 792.03it/s]

3.2446500517228856


In [15]:
best_cross_prob = cross_probs[np.argmin(np.mean(pc, axis = 1))]

In [38]:
ls = np.arange(start = 5, stop = 35, step = 5)

for l_bits in ls:

    for i in range(n_exec):
        GA = GeneticAlgorithm(n = 10, m = 20, l = l_bits, max_eval = 10000, mut_prob = best_mut_prob, cross_prob = best_cross_prob, k = 20)
        GA.solve()
        fitness.append(GA.best_fitness)
        print(GA.best_fitness)
    f_l.append(fitness.copy())
    fitness.clear()

Generations:   3%|▎         | 320/10000 [00:00<00:06, 1575.54it/s]

50.2849204378625


Generations:   1%|▏         | 148/10000 [00:00<00:06, 1469.76it/s]

50.28492043786251


Generations:   1%|▏         | 134/10000 [00:00<00:07, 1338.40it/s]

...

4.953084867374159


Generations:   0%|          | 26/10000 [00:00<00:41, 242.95it/s]

4.532787592870818


Generations: 100%|██████████| 10000/10000 [00:38<00:00, 257.20it/s]

9.856316653315318


In [39]:
best_l = ls[np.argmin(np.mean(f_l, axis = 1))]

In [40]:
ms = np.arange(start = 10, stop = 110, step = 10)

for pop_size in ms:

    for i in range(n_exec):
        GA = GeneticAlgorithm(n = 10, m = pop_size, l = best_l, max_eval = 10000, mut_prob = best_mut_prob, cross_prob = best_cross_prob, k = pop_size)
        GA.solve()
        fitness.append(GA.best_fitness)
        print(GA.best_fitness)
    f_m.append(fitness.copy())
    fitness.clear()

Generations:   0%|          | 36/10000 [00:00<00:27, 358.63it/s]

8.96180259202309


Generations:   0%|          | 42/10000 [00:00<00:24, 412.12it/s]

4.2249060475911335


Generations:   0%|          | 40/10000 [00:00<00:26, 374.64it/s]

...

7.42620915685626


Generations:   0%|          | 38/10000 [00:00<00:26, 377.45it/s]

9.356572349463477


Generations: 100%|██████████| 10000/10000 [00:20<00:00, 476.97it/s]

9.487165113916006


In [41]:
best_pop_size = ms[np.argmin(np.mean(f_m, axis = 1))]

In [42]:
with open('hyperparameters_rastrigin.npy', 'wb') as f:
    np.save(f, pm)
    np.save(f, pc)
    np.save(f, f_l)
    np.save(f, f_m)

In [43]:
# n = 10, m = 20, l = 20, max_eval = 10000, k = 20
print('Best Mutation Probability:', best_mut_prob)
print('Best Crossover Probability:', best_cross_prob)
print('Best Population Size:', best_pop_size)
print('Best L-bits:', best_l)

Best Mutation Probability: 0.03
Best Crossover Probability: 0.9000000000000002
Best Population Size: 30
Best L-bits: 20


In [45]:
GA = GeneticAlgorithm(n = 10, m = best_pop_size, l = best_l, max_eval = 10000, mut_prob = best_mut_prob, cross_prob = best_cross_prob, k = best_pop_size)
GA.solve()
print(GA.best_fitness)

Generations: 100%|██████████| 10000/10000 [00:14<00:00, 677.70it/s]

6.574952976308168


In [ ]:
# with open('hyperparameters_rastrigin.npy', 'rb') as f:
#     pm = np.load(f)
#     pc = np.load(f)
#     f_l = np.load(f)
#     f_m = np.load(f)